In [21]:
import numpy as np
import pickle
import os
import data_manager as dm
import myutils

In [ ]:
sampling_rate = 16 #Hz

In [ ]:
def resample(data, rate):
    interval = 1/rate
    duration = data[-1, 0]
    ts = np.arange(0, duration, interval)    
    count = len(ts)
    #print(interval, ", ",count)
    res = np.zeros((count-1, data.shape[1]))
    
    i = 0
    j = 0
    while i < count-1:
        t = ts[i]
        res[i, 0] = t        
        t1 = data[j, 0]
        t2 = data[j+1, 0]
        
        if t1 <= t < t2:
            factor = (t - t1)/(t2-t1);
            res[i, 1:] = (1-factor)*data[j, 1:]  + factor*data[j+1, 1:]
            i+=1
        
        while not(data[j, 0] <= ts[i] < data[j+1, 0]):
            j+=1   
            
    return res

In [ ]:
def time_to_index(data, ts):
    count = len(ts)
    ix = np.zeros(ts.shape)
    
    i = 0
    j = 0
    while i < count:        
        if data[j, 0] <= ts[i] < data[j+1, 0]:
            ix[i] = j
            i += 1
        
        if i==count:
            break;
        
        while not(data[j, 0] <= ts[i] < data[j+1, 0]):
            j += 1
            
    return ix        
    

In [ ]:
def process_annots_uva(annots, accel):
    annots[:, 0] = time_to_index(accel, annots[:, 0])
    
    for i in range(len(annots)):
        if 1 <= annots[i, 1] < 400:
            annots[i, 1] = 1
        elif 400 <= annots[i, 1] < 1000:
            annots[i, 1] = 2
        else:
            print("XXXXXXX UVA annot Problem XXXXXXXXXXX")
            
    annots = annots.astype(int)
    return annots
    

In [ ]:
def read_data_uva_lab():
    print("Reading data UVA lab")
    path = 'C:\\ASM\\DevData\\eating\\our_data\\lab_data\\'
    data = []
    for i in range(35):        
        accel = np.genfromtxt(path + "accel_"+str(i), delimiter=',')        
        print("\n\nSession ", str(i), " Before process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
        #print(accel[:10, :])
        
        accel = resample(accel, sampling_rate)
        #print(accel[:10, :])
        #accel[:, 1:] = smooth_data(accel[:, 1:], smooth_factor)
        print("Session ", str(i), " After process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
        print(accel[:10, :])
          
        annots = np.genfromtxt(path+"annot_"+str(i), delimiter=',') 
        print("Session ", str(i), " Annots >> Duration: ", str(annots[0, 0]), " - ", str(annots[-1, 0]), ", Count: "+str(len(annots)) )         
        annots = process_annots_uva(annots, accel)        
        print(annots[:10, :])
        
        dsess = [accel, annots]        
        dsubject = [dsess]        
        data.append(dsubject)
    
    dm.save_data(data, "uva_lab_data", "data")            
    return data
    

In [ ]:
data = read_data_uva_lab()
print("UVA lab Subject count: ", len(data))

In [ ]:
def read_data_uva_free():
    print("Reading data UVA free")
    path = 'C:\\ASM\\DevData\\eating\\our_data\\free_data\\'
    data = []
    for i in range(16):        
        accel = np.genfromtxt(path + "accel_"+str(i), delimiter=',')                
        print("Session ", str(i), " Before process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
        #print(accel[:10, :])
        
        accel = resample(accel, sampling_rate)
        #print(accel[:10, :])
        #accel[:, 1:] = smooth_data(accel[:, 1:], smooth_factor)
        print("Session ", str(i), " After process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
        print(accel[:10, :])
          
        annots = [] 
        
        dsess = [accel, annots]        
        dsubject = [dsess]        
        data.append(dsubject)
    
    dm.save_data(data, "uva_free_data", "data")
        
    return data

In [ ]:
data = read_data_uva_free()
print("UVA free Subject count: ",len(data))

In [ ]:
def process_bite_label_steven(label):
    label = label.replace("P","1").replace("D","2").replace("I","3").replace("Q","4")
    label = label.replace("N","5").replace("C","6").replace("S","7").replace("M","8")
    label = label.replace("R","1").replace("L","2").replace("X","-1")
    return int(label)

def process_bite_annots_steven(annots, accel):    
    a = np.zeros((len(annots), 2))
    for i in range(len(annots)):
        s = annots[i].split(",")
        t1 = float(s[0].rstrip())
        t2 = float(s[1].rstrip())
        label = process_bite_label_steven(s[2].rstrip())
        a[i, 0] = t1+t2/2
        a[i, 1] = label
    
    res = []
    hand = 0    
    last_was_bite = False
    for i in range(len(a)):
        #print(a[i, 0], ", ", a[i, 1])
        if a[i, 1]==21 or a[i,1] ==22:            
            if last_was_bite:    
                if a[i, 1]==21:
                    hand = 1
                else:
                    hand = 2
            else:
                if hand ==1 and a[i, 1]==21 or hand ==2 and a[i, 1]==22:
                    print( "************************ Similar hand found twice ****************")
                
                hand = 3
            
            last_was_bite = False
            
        elif a[i, 1]==3 or a[i, 1]==4:
            if hand==0:
                print("********** Hand is not found : " , annots[i] , ", line no: ", (i + 1))                
                continue
            
            r = [0, 0]
            r[0] = a[i, 0]
            if a[i, 1]==3:
                r[1] = 1
            else:
                r[1] = 2
            
            if hand ==1 or hand ==3:
                res.append(r)
            
            if last_was_bite:
                print("************************ Bite repeated **************** ", i)
            last_was_bite = True
        
    #print(res)            
    res = np.array(res)
    #print(res.shape)
    res[:, 0] = time_to_index(accel, res[:, 0])
    res = res.astype(int)
    return res

In [ ]:
def read_data_steven_lab():
    path = 'C:\\ASM\\PublicData\\ACE_lab_public_2\\lab_public\\'
    
    data = [];
    for subject in range(7):
        dsubject = [];
        for sess in range(2):
            if (subject==0 and sess==0) or (subject==1 and sess==1):
                continue
            
            filePathAccel = path + "\\0" + str(subject) + "\\000" + str(sess) + "\\watch_right_000" + str(sess) + ".csv";
            filePathAnnots = path + "\\0" + str(subject) + "\\000" + str(sess) + "\\annot_events.csv";
            
            accel = np.genfromtxt(filePathAccel, delimiter=',')
            accel = accel[:, :4]
            accel[:, 0] = accel[:, 0]/1e9
            print("Subject ", str(subject), ", Session ", str(sess), " Before process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
            #print(accel[:10, :])
        
            accel = resample(accel, sampling_rate)            
            #accel[:, 1:] = smooth_data(accel[:, 1:], smooth_factor)
            print("Subject ", str(subject), ", Session ", str(sess), " After process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
            print(accel[:5, :])
          
            file = open(filePathAnnots)
            annots = file.readlines()
            annots = process_bite_annots_steven(annots, accel)        
            print("Subject ", str(subject), ", Session ", str(sess), " Annots >> Duration: ", str(annots[0, 0]), " - ", str(annots[-1, 0]), ", Count: "+str(len(annots)) )         
            print(annots[:5, :])
            
            dsess = [accel, annots]        
            dsubject.append(dsess)        
            
        data.append(dsubject)
    
    dm.save_data(data, "steven_lab_data", "data")
    
    return data
    
    

In [ ]:
data = read_data_steven_lab()
print("Steven lab Subject count: ",len(data))

In [ ]:
def get_meal_code(meal):    
    code = -1
    if meal == "meal" or meal=="lunch" or meal=="dinner":
        code = 1
    elif meal=="snack":
        code = 2    
    elif meal=="drink":
        code = 3
    return code;

def process_meal_annots_steven(annots, accel, subject_code, session):
    meal_count = len(annots)
    accel_count = len(accel)
    if subject_code == 5 and session == 0:
        meal_count = 9
    elif subject_code == 6 and session == 0:
        meal_count = 2
    elif subject_code == 102 and session == 1:
        meal_count = 1
    elif subject_code == 104 and session == 0:
        meal_count = 7
    
    j = 0
    a = np.zeros((meal_count, 3))
    for i in range(meal_count):        
        s = annots[i].split(",")
        t1 = float(s[1].rstrip())
        t2 = float(s[2].rstrip())
        a[i, 2] = get_meal_code(s[3].rstrip().strip())
        
        if a[i, 2]==-1:
            print(" \n\n******************* Meal code problem *************** code is -1\n\n")
        
        t = t1
        while j<accel_count-1:
            if accel[j, 0] <= t < accel[j+1, 0]:            
                a[i, 0] = j
                break            
            j+=1
            
        t = t2
        while j<accel_count-1:
            if accel[j, 0] <= t < accel[j+1, 0]:            
                a[i, 1] = j
                break            
            j+=1
        
        if a[i][1] == 0:
            if subject_code == 107 and session == 0 or subject_code == 107 and session == 2:
                a[i, 1] = accel_count - 1
        
        if a[i, 0] == 0 or a[i, 1] == 0:
            print("***** Time indexing problem. Meal line: " , (i + 1))
        
    return a

In [ ]:
def read_data_steven_free():
    path = 'C:\\ASM\\PublicData\\PHI_ACE-E-FL_public\\'
    subject_codes = [2, 3, 4, 5, 6, 101, 102, 103, 104, 107, 109]
    
    data = [];
    for subject in range(len(subject_codes)):
        dsubject = [];
        sess_count = 2
        if subject_codes[subject] == 107:
            sess_count = 5
        
        for sess in range(sess_count):
            if subject_codes[subject] < 10:
                filePathAccel = path + "\\0" + str(subject_codes[subject]) + "\\000" + str(sess) + "\\watch_right_000" + str(sess) + ".csv";
                filePathAnnots = path + "\\0" + str(subject_codes[subject]) + "\\000" + str(sess) + "\\meal_events.csv";
            elif subject_codes[subject] == 109:
                filePathAccel = path + "\\" + str(subject_codes[subject]) + "\\000" + str(sess+3) + "\\watch_right_000" + str(sess+3) + ".csv";
                filePathAnnots = path + "\\" + str(subject_codes[subject]) + "\\000" + str(sess+3) + "\\meal_events.csv";
            else:
                filePathAccel = path + "\\" + str(subject_codes[subject]) + "\\000" + str(sess) + "\\watch_right_000" + str(sess) + ".csv";
                filePathAnnots = path + "\\" + str(subject_codes[subject]) + "\\000" + str(sess) + "\\meal_events.csv";
          
            accel = np.genfromtxt(filePathAccel, delimiter=',')
            accel = accel[:, :4]
            accel[:, 0] = accel[:, 0]/1e9
            print("\n\nSubject ", str(subject), ", Session ", str(sess), " Before process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
            #print(accel[:10, :])
        
            accel = resample(accel, sampling_rate)            
            #accel[:, 1:] = smooth_data(accel[:, 1:], smooth_factor)
            print("Subject ", str(subject), ", Session ", str(sess), " After process >> Duration: ", str(accel[0, 0]), " - ", str(accel[-1, 0]), ", Count: "+str(len(accel)) )         
            #print(accel[:10, :])
          
            file = open(filePathAnnots)
            annots = file.readlines()
            annots = process_meal_annots_steven(annots, accel, subject_codes[subject], sess)        
            print("Subject ", str(subject), ", Session ", str(sess), " Annots >> Duration: ", str(annots[0, 0]), " - ", str(annots[-1, 0]), ", Count: "+str(len(annots)) )         
            print(annots)
            
            dsess = [accel, annots]        
            dsubject.append(dsess)        
            
        data.append(dsubject)
    
    dm.save_data(data, "steven_free_data", "data")
    
    return data
    

In [ ]:
data = read_data_steven_free()
print("Steven Free Subject count: ",len(data))

In [22]:
#smooth and save data
factors = [800, 825, 850, 875, 900, 925, 950]
ds_names = ["uva_lab_data", "uva_free_data", "steven_lab_data", "steven_free_data"]

for dsn in ds_names:    
    for f in factors:    
        data = dm.get_data(dsn, "data")
        fact = f/1000
        print(dsn, " : ", fact)
        data = myutils.smooth_dataset(data, fact)
        dm.save_data(data, dsn+"_"+str(f), "data")


uva_lab_data  :  0.8
Smothing data with factor  0.8  ...
Subject  0 , Session  0
Subject  1 , Session  0
Subject  2 , Session  0
Subject  3 , Session  0
Subject  4 , Session  0
Subject  5 , Session  0
Subject  6 , Session  0
Subject  7 , Session  0
Subject  8 , Session  0
Subject  9 , Session  0
Subject  10 , Session  0
Subject  11 , Session  0
Subject  12 , Session  0
Subject  13 , Session  0
Subject  14 , Session  0
Subject  15 , Session  0
Subject  16 , Session  0
Subject  17 , Session  0
Subject  18 , Session  0
Subject  19 , Session  0
Subject  20 , Session  0
Subject  21 , Session  0
Subject  22 , Session  0
Subject  23 , Session  0
Subject  24 , Session  0
Subject  25 , Session  0
Subject  26 , Session  0
Subject  27 , Session  0
Subject  28 , Session  0
Subject  29 , Session  0
Subject  30 , Session  0
Subject  31 , Session  0
Subject  32 , Session  0
Subject  33 , Session  0
Subject  34 , Session  0
uva_lab_data  :  0.825
Smothing data with factor  0.825  ...
Subject  0 , Sess